<a href="https://colab.research.google.com/github/Epistimonas/imagenes/blob/master/LabAnalisisdeTexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, GlobalAveragePooling1D

import numpy as np
import string
import pandas as pd
import matplotlib.pylab as plt
import os, re, json, functools

#dark mode
plt.rc_context({'xtick.color':'w', 'ytick.color':'w', 'text.color':'w', 'axes.labelcolor':'w'})

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

TensorFlow 2.x selected.


In [0]:
pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [0]:
os.environ['KAGGLE_USERNAME'] = "fermarro13"
os.environ['KAGGLE_KEY'] = "1497a0fab0c3b60b7bba56211f41ca04"

In [0]:
!kaggle datasets download shivamb/netflix-shows

  0% 0.00/971k [00:00<?, ?B/s]
100% 971k/971k [00:00<00:00, 63.4MB/s]


In [0]:
!unzip -o "netflix-shows.zip"

Archive:  netflix-shows.zip
  inflating: netflix_titles.csv      


In [0]:
os.listdir()

['.config', 'netflix-shows.zip', 'netflix_titles.csv', 'sample_data']

In [0]:
shows = pd.read_csv('netflix_titles.csv')
shows.head

<bound method NDFrame.head of        show_id  ...                                        description
0     81145628  ...  Before planning an awesome wedding for his gra...
1     80117401  ...  Jandino Asporaat riffs on the challenges of ra...
2     70234439  ...  With the help of three human allies, the Autob...
3     80058654  ...  When a prison ship crash unleashes hundreds of...
4     80125979  ...  When nerdy high schooler Dani finally attracts...
...        ...  ...                                                ...
6229  80000063  ...  This parody of first-person shooter games, mil...
6230  70286564  ...  Marc Maron stars as Marc Maron, who interviews...
6231  80116008  ...  Nursery rhymes and original music for children...
6232  70281022  ...  Set during the Russian Revolution, this comic ...
6233  70153404  ...  This hit sitcom follows the merry misadventure...

[6234 rows x 12 columns]>

In [0]:
shows.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [0]:
shows.listed_in

0                      Children & Family Movies, Comedies
1                                         Stand-Up Comedy
2                                                Kids' TV
3                                                Kids' TV
4                                                Comedies
                              ...                        
6229    TV Action & Adventure, TV Comedies, TV Sci-Fi ...
6230                                          TV Comedies
6231                                               Movies
6232             British TV Shows, TV Comedies, TV Dramas
6233                       Classic & Cult TV, TV Comedies
Name: listed_in, Length: 6234, dtype: object

In [0]:
shows.description

0       Before planning an awesome wedding for his gra...
1       Jandino Asporaat riffs on the challenges of ra...
2       With the help of three human allies, the Autob...
3       When a prison ship crash unleashes hundreds of...
4       When nerdy high schooler Dani finally attracts...
                              ...                        
6229    This parody of first-person shooter games, mil...
6230    Marc Maron stars as Marc Maron, who interviews...
6231    Nursery rhymes and original music for children...
6232    Set during the Russian Revolution, this comic ...
6233    This hit sitcom follows the merry misadventure...
Name: description, Length: 6234, dtype: object

In [0]:
description = shows['description']
genres = shows['listed_in']

dataset = shows[['description', 'listed_in']] 

In [0]:
len(dataset)

6234

In [0]:
CLASS_NAMES = ['Drama','Comedy','Thriller','Romance','Action']
CLASS_COUNT = len(CLASS_NAMES)
CLASS_COUNT

5

In [0]:
dataset.iloc[0]['listed_in']

'Children & Family Movies, Comedies'

In [0]:
dataset['listed_in'].replace("\'", "\"", regex=True, inplace=True)
dataset.iloc[0]['listed_in']

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


'Children & Family Movies, Comedies'

In [0]:
label_count = dict()

def genre_to_list(str_genre):
  keep = True
  genre_list = list(map(lambda genre:genre['name'],json.loads(str_genre)))
  
  for genre in genre_list:
    count = label_count.get(genre, 0) + 1
    label_count[genre] = count
    if (count > 500):
      keep = False
    #if (genre_to_id.get(genre,-1) == -1):
    #  genre_to_id[genre] = len(genre_to_id.keys())
  genres_as_ids = list(map(lambda x: CLASS_NAMES.index(x), filter(lambda x: x in CLASS_NAMES, genre_list)))
  #genres_as_ids = list(map(lambda x:genre_to_id[x], genre_list)) 
  genres_one_hot = list()

  if (len(genres_as_ids) > 0 and keep):
    genres_one_hot = tf.keras.utils.to_categorical(genres_as_ids, CLASS_COUNT, dtype='int')
    genres_one_hot = functools.reduce(lambda x,y : x+y,genres_one_hot)

  return genres_one_hot

In [0]:
dataset['listed_in'] = dataset['listed_in'].apply(lambda x: genre_to_list(x))
dataset.iloc[0]['listed_in']

JSONDecodeError: ignored